In [1]:
# activate line execution
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
import numpy as np
import logging
logging.basicConfig(level=logging.INFO)

In [3]:
from MulticoreTSNE import MulticoreTSNE as TSNE
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [4]:
import os
from hydra import initialize, initialize_config_module, initialize_config_dir, compose
from omegaconf import OmegaConf

initialize(version_base=None, config_path="configs")
cfg = compose(config_name="config.yaml")

hydra.initialize()

/mlodata1/hokarami/Machine-Learning-Collection/ML/Pytorch/GANs/4. WGAN-GP/test2/lib/python3.8/site-packages/hydra/_internal/defaults_list.py:251: UserWarning: In 'config.yaml': Defaults list is missing `_self_`. See https://hydra.cc/docs/1.2/upgrades/1.0_to_1.1/default_composition_order for more information
  warnings.warn(msg, UserWarning)


In [5]:
from dotenv import load_dotenv
import wandb
import os

load_dotenv()
wandb.login(key=os.getenv("WANDB_KEY"))

api = wandb.Api()

True

ERROR:wandb.jupyter:Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: g-hojatkarami (hokarami). Use `wandb login --relogin` to force relogin


True

In [6]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader

from tqdm import tqdm

# Load Data

In [7]:
from Utils.data import load_data, Physio3
print(cfg.data)

SPLIT=0
path_train, path_val = cfg.data.path_train.format(SPLIT=1), cfg.data.path_eval.format(SPLIT=1)
train_dataset, val_dataset, train_schema, val_schema = load_data(path_train, path_val, batch_size = cfg.bs)

{'name': 'p12', 'static_variables': None, 'timeseries_variables': None, 'path_raw': '/mlodata1/hokarami/tedam/raw', 'path_processed': '/mlodata1/hokarami/tedam/TimEHR/data/processed/p12-real', 'path_train': '/mlodata1/hokarami/tedam/TimEHR/data/processed/p12-real/split{SPLIT}/train.pkl', 'path_eval': '/mlodata1/hokarami/tedam/TimEHR/data/processed/p12-real/split{SPLIT}/eval.pkl', 'img_size': 64, 'd_static': 9, 'd_ts': None, 'label': None}


# Train/Load Model

In [8]:
from Utils.models import TimEHR

model = TimEHR(cfg)

/mlodata1/hokarami/Machine-Learning-Collection/ML/Pytorch/GANs/4. WGAN-GP/test2/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


[KeOps] Compiling cuda jit compiler engine ... 
[KeOps] Warning : There were warnings or errors :
g++: error: WGAN-GP/test2/lib/python3.8/site-packages/keopscore: No such file or directory
g++: error: /mlodata1/hokarami/Machine-Learning-Collection/ML/Pytorch/GANs/4.: No such file or directory
g++: error: WGAN-GP/test2/lib/python3.8/site-packages/keopscore/binders/nvrtc/nvrtc_jit.cpp: No such file or directory

OK
[pyKeOps] Compiling nvrtc binder for python ... 
[KeOps] Warning : There were warnings or errors :
/bin/sh: 1: /mlodata1/hokarami/Machine-Learning-Collection/ML/Pytorch/GANs/4.: not found
g++: error: WGAN-GP/test2/lib/python3.8/site-packages/keopscore: No such file or directory
g++: error: /mlodata1/hokarami/Machine-Learning-Collection/ML/Pytorch/GANs/4.: No such file or directory
g++: error: WGAN-GP/test2/lib/python3.8/site-packages/pykeops/common/keops_io/pykeops_nvrtc.cpp: No such file or directory

OK
Generator has 1,304,964 trainable parameters
Critic has 212,177 trainabl

In [9]:
# model.from_pretrained(path_cwgan='hokarami/CWGAN/uj5gf643',path_pix2pix='hokarami/PIXGAN/lorajx1i')
model.from_pretrained(path_cwgan='hokarami/CWGAN/uj5gf643',path_pix2pix='hokarami/PIXGAN/9hec4rye')


INFO:root:Loading CWGAN from run: hokarami/CWGAN/uj5gf643


CWGAN - run_path:  hokarami/CWGAN/uj5gf643


INFO:root:Loading PIXGAN from run: hokarami/PIXGAN/9hec4rye


run is old
last_epoch:  400
PIXGAN - run_path:  hokarami/PIXGAN/9hec4rye
last_epoch:  150


In [10]:
model.cwgan.run_path
model.cwgan.epoch_no

model.pix2pix.run_path
model.pix2pix.epoch_no

model.progress

'hokarami/CWGAN/uj5gf643'

400

'hokarami/PIXGAN/9hec4rye'

150

{}

# Generation

In [11]:
from Utils.utils import mat2df
# get synthetic data
fake_static, fake_data = model.generate(train_dataset, train_schema, count=len(train_dataset), method='real')
df_ts_fake, df_demo_fake = mat2df(fake_data,fake_static, train_schema)

# get train data
train_static, train_data = model._get_data(train_dataset)
df_ts_train, df_demo_train = mat2df(train_data,train_static, train_schema)

# get test data
test_static, test_data = model._get_data(val_dataset)
df_ts_test, df_demo_test = mat2df(test_data,test_static, val_schema)

INFO:root:Using static data from the training set (9589 samples)
  0%|          | 0/150 [00:00<?, ?it/s]

100%|██████████| 38/38 [00:00<00:00, 277.81it/s]


# Evaluation

In [31]:
from Utils.evaluation import evaluate

inputs = {

    # normalized data
    'fake_static': fake_static,
    'fake_data': fake_data,
    'train_static': train_static,
    'train_data': train_data,
    'test_static': test_static,
    'test_data': test_data,

    # dataframes from normalized data
    'df_ts_fake': df_ts_fake,
    'df_demo_fake': df_demo_fake,
    'df_ts_train': df_ts_train,
    'df_demo_train': df_demo_train,
    'df_ts_test': df_ts_test,
    'df_demo_test': df_demo_test,

    'state_vars': train_schema.dynamic_processor['mean'].index.tolist(),
}

In [32]:
fake_data.shape, train_data.shape, test_data.shape

((9589, 2, 64, 64), (9589, 2, 64, 64), (2398, 2, 64, 64))

In [34]:
wandb_task_name = f"NOTEBOOK-{cfg.data.name}-s{cfg.split}"

evaluate(inputs, wandb_task_name)

# You can now view the results in the wandb dashboard under USERNAME/TimEHR_Results
# The results are also available locally in Results/{wandb_task_name}/wandb/{run_name}/

[info] Preprocessing 
[info] Preprocessing 
[info] Preprocessing 
[info] Preprocessing 

[info] TRTR
[LightGBM] [Info] Number of positive: 1346, number of negative: 8243
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.014723 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 28499
[LightGBM] [Info] Number of data points in the train set: 9589, number of used features: 175
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.140369 -> initscore=-1.812227
[LightGBM] [Info] Start training from score -1.812227
AUROC: 0.8316599964371048
AUPRC: 0.48931353244748893
F1: 0.4992743105950654

[info] TSTR
[LightGBM] [Info] Number of positive: 1355, number of negative: 8234
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.011836 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 41447
[LightGBM] [Info] Number of data points in the trai

Adv ACC/AA_test_syn,▁
Adv ACC/AA_train_syn,▁
Adv ACC/AA_train_test,▁
Adv ACC/NNAA,▁
MIA/JSD,▁
MIA/WD,▁
MIA/knn-auroc,▁
Synthcity[Train-Synthetic]/privacy.identifiability_score.score,▁
Synthcity[Train-Synthetic]/privacy.identifiability_score.score_OC,▁
Synthcity[Train-Synthetic]/stats.prdc.coverage,▁
Synthcity[Train-Synthetic]/stats.prdc.density,▁
